In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import math
import pickle
import random

import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pymesh

sys.path.append("../../")

import implicitmodules.torch as dm

In [ ]:
seed = 1337
refinement_order = 2
sigma = 2.
sphere_count = 5
spheres = []
spheres_data = []

random.seed(seed)

sphere_mesh = pymesh.generate_icosphere(0.5, [0., 0., 0.], refinement_order)
sphere_points = torch.tensor(sphere_mesh.vertices, dtype=torch.get_default_dtype())
sphere_triangles = torch.tensor(sphere_mesh.faces, dtype=torch.long)

# axis = random.randint(0, 2)
axis = 0
scale = random.gauss(0., sigma)
scale_matrix = torch.eye(3)
scale_matrix[axis, axis] = scale
rot_matrix = torch.eye(3)
rot_matrix[0:2, 0:2] = dm.Utilities.rot2d(math.pi/3.)
trans_matrix = rot_matrix @ scale_matrix
rot_matrix = torch.eye(3)
rot_matrix[1:3, 1:3] = dm.Utilities.rot2d(2.*math.pi/3.)
trans_matrix = rot_matrix @ trans_matrix

source_points = sphere_points
source_triangles = sphere_triangles
target_points = dm.Utilities.linear_transform(sphere_points, trans_matrix)
target_triangles = sphere_triangles

source_deformable = dm.Models.DeformableMesh(source_points, source_triangles)
target_deformable = dm.Models.DeformableMesh(target_points, target_triangles)

print("Vertices count={vertices_count}".format(vertices_count=sphere_points.shape[0]))

In [ ]:
%matplotlib qt5
ax = plt.subplot(projection='3d')
ax.plot_trisurf(source_points[:, 0].numpy(), source_points[:, 1].numpy(), source_points[:, 2].numpy(), triangles=source_triangles)
ax.plot_trisurf(target_points[:, 0].numpy(), target_points[:, 1].numpy(), target_points[:, 2].numpy(), triangles=target_triangles, color= (0,1,0,0.2), edgecolor=(0, 1, 0, 0.2))
dm.Utilities.set_aspect_equal_3d(ax)
plt.show()

In [ ]:
sigma = 0.5
aabb = dm.Utilities.AABB.build_from_points(torch.tensor(sphere_mesh.vertices))

implicit1_points = aabb.fill_uniform_density(50)
implicit1_rot = torch.eye(3).repeat(implicit1_points.shape[0], 1, 1)

# implicit1_c = 0.2*torch.randn(implicit1_points.shape[0], 3, 1) + 1.
# implicit1_c.requires_grad_()

# implicit1_c = torch.zeros(implicit1_points.shape[0], 3, 3)
# implicit1_c[:, 0, 0] = 1.
# implicit1_c[:, 1, 1] = 1.
# implicit1_c[:, 2, 2] = 1.
implicit1_c = torch.zeros(implicit1_points.shape[0], 3, 1)
implicit1_c[:, 0, 0] = 1.

angles = torch.zeros(implicit1_points.shape[0], 3, requires_grad=True)

In [ ]:
global_translation = dm.DeformationModules.GlobalTranslation(3)

implicit1 = dm.DeformationModules.ImplicitModule1(3, implicit1_points.shape[0], sigma, implicit1_c, nu=0.1, gd=(implicit1_points, implicit1_rot))

In [ ]:
def precompute(init_manifold, modules, parameters):
    rot_x = dm.Utilities.rot3d_x_vec(parameters['angles']['params'][0][:, 0])
    rot_y = dm.Utilities.rot3d_y_vec(parameters['angles']['params'][0][:, 1])
    rot_z = dm.Utilities.rot3d_z_vec(parameters['angles']['params'][0][:, 2])
    basis = torch.einsum('nik, nkl, nlj->nij', rot_z, rot_y, rot_x)
    print(angles[4])
    print(basis[4])
    init_manifold[2].gd = (init_manifold[2].gd[0], basis)


sigmas_varifold = [0.5]
attachment = dm.Attachment.VarifoldAttachment(3, sigmas_varifold)

model = dm.Models.RegistrationModel([source_deformable], [global_translation, implicit1], [attachment], lam=10000., precompute_callback=precompute, other_parameters={'angles': {'params': [angles]}})

In [ ]:
shoot_solver = 'euler'
shoot_it = 10

costs = {}
fitter = dm.Models.Fitter(model, optimizer='torch_lbfgs')

fitter.fit(target_deformable, 50, costs=costs, options={'shoot_solver': shoot_solver, 'shoot_it': shoot_it, 'line_search_fn': 'strong_wolfe'})

In [ ]:
intermediates = {}
deformed = model.compute_deformed(shoot_solver, shoot_it, intermediates=intermediates)[0][0].detach()

In [ ]:
print(model.init_manifold[2].gd[1])

In [ ]:
print(angles)

In [ ]:
%matplotlib qt5
plt.plot(range(angles[:, 0].shape[0]), angles[:, 2].detach().numpy())
plt.show()

In [ ]:
print(intermediates['controls'])

In [ ]:
%matplotlib qt5
ax = plt.subplot(projection='3d')
ax.plot_trisurf(target_points[:, 0].numpy(), target_points[:, 1].numpy(), target_points[:, 2].numpy(), triangles=source_triangles)
ax.plot_trisurf(deformed[:, 0].numpy(), deformed[:, 1].numpy(), deformed[:, 2].numpy(), triangles=source_triangles, color= (0,1,0,0.2), edgecolor=(0, 1, 0, 0.2))
dm.Utilities.set_aspect_equal_3d(ax)
plt.show()